In [2]:
# tensorflow==2.15
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import pandas as pd
import numpy as np
# scikit-learn
# tensorboard
import matplotlib as mt
# streamlit

In [3]:
d=pd.read_csv(r'C:\Users\nsri3\OneDrive\Desktop\ra\sessin53\Churn_Modelling.csv')

In [4]:
d.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# drop unnessesary features
d=d.drop(['RowNumber','CustomerId','Surname'],axis=1)
d.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# encode character data->geography,gender
lable_gender=LabelEncoder()
d['Gender']=lable_gender.fit_transform(d['Gender'])
d.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
from sklearn.preprocessing import OneHotEncoder
onehot_geo=OneHotEncoder()
geo_encoder=onehot_geo.fit_transform(d[['Geography']])

In [8]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_geo.get_feature_names_out(['Geography']))

In [9]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
# combine onehot cols to real data
d=pd.concat([d.drop('Geography',axis=1),geo_encoder_df],axis=1)
d.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
with open('lable_encoder_gender.pkl','wb') as file:
    pickle.dump(lable_gender,file)
with open('onehot_encode_geo.pkl','wb') as file:
    pickle.dump(onehot_geo,file)


In [12]:
# devide data to independent and dependent

x=d.drop('Exited',axis=1)
y=d['Exited']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
scalar=StandardScaler()
xtrain=scalar.fit_transform(xtrain)
xtest=scalar.transform(xtest)

In [13]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scalar,file)
# with open('xtest.pkl','wb') as file:
#     pickle.dump(xtest,file)


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [16]:
# build ann model
model=Sequential([
    Dense(64,activation='relu',input_shape=(xtrain.shape[1],)),#1st hiiden layer
    Dense(32,activation='relu'),#hiddenlayer2
    Dense(1,activation='sigmoid')#output layer
])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()

In [20]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [32]:
#set upt tensorboard
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [33]:
# setup earlystopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [34]:
# train model
history=model.fit(
    xtrain,ytrain,validation_data=(xtest,ytest),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3144 - accuracy: 0.8696 - val_loss: 0.3500 - val_accuracy: 0.8575
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3081 - accuracy: 0.8727 - val_loss: 0.3488 - val_accuracy: 0.8575
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3082 - accuracy: 0.8720 - val_loss: 0.3668 - val_accuracy: 0.8530
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3076 - accuracy: 0.8673 - val_loss: 0.3514 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3042 - accuracy: 0.8735 - val_loss: 0.3575 - val_accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3025 - accuracy: 0.8726 - val_loss: 0.3520 - val_accuracy: 0.8595
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2995 - accuracy: 0.8750 - val_loss: 0.3578 - val_accuracy: 0.8565

In [36]:
model.save('model.h5')

In [37]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [38]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 17916), started 0:05:08 ago. (Use '!kill 17916' to kill it.)

In [ ]:
# load the pickle file
